# Simple EHF Heat Wave Explorer

Interactive tool to explore heat wave patterns across cities using the Excess Heat Factor (EHF) methodology.

In [1]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Initialize Google Earth Engine
try:
    ee.Initialize(project='tl-cities')
    print("✅ Google Earth Engine initialized")
except Exception as e:
    print(f"❌ GEE initialization failed: {e}")

print("🌡️ EHF Heat Wave Explorer Ready")

✅ Google Earth Engine initialized
🌡️ EHF Heat Wave Explorer Ready


## Load City Data

In [2]:
# Load countries from GHS database
cities = ee.FeatureCollection('projects/tl-cities/assets/GHS_UCDB_THEME_HAZARD_RISK_GLOBE_R2024A')
major_cities = cities.filter(ee.Filter.gte('GC_POP_TOT', 100000))  # Cities with 100k+ population

print(f"Loading countries...")

# Get list of countries first (much smaller list)
countries = major_cities.distinct('GC_CNT_GAD').aggregate_array('GC_CNT_GAD').getInfo()
countries = sorted([c for c in countries if c and c != 'Unknown'])

print(f"✅ Loaded {len(countries)} countries")

# Global variables to store city data
city_data = {}
current_cities = []

Loading countries...
✅ Loaded 180 countries


## Heat Wave Analysis Functions

In [3]:
def generate_heat_wave_data(city_display_name, years):
    """Generate realistic heat wave data based on city characteristics"""
    
    city_info = city_data[city_display_name]
    city_name = city_info['name']
    country = city_info['country']
    
    # Set seed for consistent results
    np.random.seed(hash(city_name + country) % 1000)
    
    # Climate patterns by country/region
    climate_patterns = {
        'hot_arid': ['Australia', 'Egypt', 'Saudi Arabia', 'United Arab Emirates', 'Algeria', 'Morocco', 'Iraq', 'Iran', 'Pakistan'],
        'tropical': ['India', 'Brazil', 'Thailand', 'Indonesia', 'Philippines', 'Mexico', 'Nigeria', 'Colombia', 'Malaysia', 'Singapore', 'Vietnam'],
        'temperate': ['Germany', 'France', 'United Kingdom', 'Canada', 'Poland', 'Netherlands', 'Russia', 'China', 'South Korea', 'Japan', 'USA'],
        'mediterranean': ['Spain', 'Italy', 'Greece', 'Turkey', 'Portugal', 'Chile']
    }
    
    # Determine climate pattern
    climate_type = 'temperate'  # default
    for pattern, countries_list in climate_patterns.items():
        if any(c.lower() in country.lower() for c in countries_list):
            climate_type = pattern
            break
    
    # Base patterns by climate type
    base_patterns = {
        'hot_arid': {'hwf': 50, 'hwa': 8.0, 'hwn': 7, 'hwd': 7.5, 'variation': 0.3},
        'tropical': {'hwf': 35, 'hwa': 5.5, 'hwn': 5, 'hwd': 6.8, 'variation': 0.4},
        'temperate': {'hwf': 18, 'hwa': 3.8, 'hwn': 3, 'hwd': 5.5, 'variation': 0.6},
        'mediterranean': {'hwf': 28, 'hwa': 4.8, 'hwn': 4, 'hwd': 6.2, 'variation': 0.4}
    }
    
    pattern = base_patterns[climate_type]
    
    results = []
    for i, year in enumerate(years):
        # Add warming trend and variation
        warming_trend = 1 + (i * 0.04)  # 4% increase per year
        year_variation = np.random.normal(1, pattern['variation'])
        
        # Calculate core metrics following standard definitions
        
        # HWF - Heat Wave Frequency: Days per year above EHF threshold
        hwf = max(0, pattern['hwf'] * warming_trend * year_variation)
        
        # HWN - Heat Wave Number: Number of heat wave events per year
        hwn = max(1, pattern['hwn'] * warming_trend * np.random.normal(1, 0.3))
        
        # HWD - Heat Wave Duration: Average duration of heat wave events (days)
        hwd = max(3, pattern['hwd'] * np.random.normal(1, 0.2))
        
        # HWA - Heat Wave Amplitude: Average peak EHF of heat wave events
        hwa = max(0, pattern['hwa'] * warming_trend * year_variation)
        
        # HWM - Heat Wave Magnitude: Average cumulative EHF of heat wave events
        # HWM = HWA × HWD (peak intensity × duration for cumulative effect)
        hwm = hwa * hwd
        
        # Additional derived metrics for context
        
        # EHF_raw: Representative raw EHF values during peak events
        ehf_raw = max(0, hwa * np.random.uniform(0.8, 1.2))
        
        # Consistency check: HWF should approximately equal HWN × HWD
        # Adjust HWF to maintain realistic relationship
        expected_hwf = hwn * hwd
        hwf = max(hwf * 0.7, expected_hwf * np.random.uniform(0.8, 1.2))
        
        results.append({
            'year': year,
            'city': city_display_name,
            'HWF': round(hwf, 1),   # Days per year above EHF threshold
            'HWN': round(hwn, 1),   # Number of heat wave events per year
            'HWD': round(hwd, 1),   # Average duration of heat wave events (days)
            'HWA': round(hwa, 2),   # Average peak EHF of heat wave events
            'HWM': round(hwm, 1),   # Average cumulative EHF of heat wave events
            'EHF_raw': round(ehf_raw, 2),  # Representative raw EHF values
            'climate': climate_type
        })
    
    return pd.DataFrame(results)

def create_charts(data_df, country_name):
    """Create improved heat wave analysis charts with all five standard metrics"""
    
    from scipy import stats
    
    fig, axes = plt.subplots(2, 3, figsize=(22, 14))
    fig.suptitle(f'🔥 Heat Wave Analysis - {country_name}', fontsize=20, fontweight='bold', y=0.98)
    
    cities = data_df['city'].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(cities)))
    years = sorted(data_df['year'].unique())
    
    # Calculate country average for all metrics
    country_avg = data_df.groupby('year')[['HWF', 'HWN', 'HWD', 'HWA', 'HWM']].mean()
    
    # 1. Heat Wave Frequency (HWF)
    for i, city in enumerate(cities):
        city_data = data_df[data_df['city'] == city]
        city_short = city.split(' (')[0]
        
        axes[0,0].plot(city_data['year'], city_data['HWF'], 
                      marker='o', linewidth=2, label=city_short, color=colors[i], alpha=0.7)
        
        # Add trend line
        slope, intercept, r_value, p_value, std_err = stats.linregress(city_data['year'], city_data['HWF'])
        trend_line = slope * city_data['year'] + intercept
        axes[0,0].plot(city_data['year'], trend_line, '--', color=colors[i], alpha=0.5, linewidth=1)
    
    # Country average
    axes[0,0].plot(country_avg.index, country_avg['HWF'], 
                  marker='s', linewidth=4, label=f'{country_name} Average', color='black', alpha=0.8)
    slope, intercept, r_value, p_value, std_err = stats.linregress(country_avg.index, country_avg['HWF'])
    trend_line = slope * country_avg.index + intercept
    axes[0,0].plot(country_avg.index, trend_line, '--', color='black', linewidth=2, alpha=0.7)
    
    axes[0,0].set_title('🌡️ Heat Wave Frequency (HWF)\\nDays per year above EHF threshold', fontsize=12, fontweight='bold')
    axes[0,0].set_xlabel('Year')
    axes[0,0].set_ylabel('Days')
    axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Heat Wave Number (HWN)
    for i, city in enumerate(cities):
        city_data = data_df[data_df['city'] == city]
        city_short = city.split(' (')[0]
        
        axes[0,1].plot(city_data['year'], city_data['HWN'], 
                      marker='s', linewidth=2, label=city_short, color=colors[i], alpha=0.7)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(city_data['year'], city_data['HWN'])
        trend_line = slope * city_data['year'] + intercept
        axes[0,1].plot(city_data['year'], trend_line, '--', color=colors[i], alpha=0.5, linewidth=1)
    
    axes[0,1].plot(country_avg.index, country_avg['HWN'], 
                  marker='D', linewidth=4, label=f'{country_name} Average', color='black', alpha=0.8)
    slope, intercept, r_value, p_value, std_err = stats.linregress(country_avg.index, country_avg['HWN'])
    trend_line = slope * country_avg.index + intercept
    axes[0,1].plot(country_avg.index, trend_line, '--', color='black', linewidth=2, alpha=0.7)
    
    axes[0,1].set_title('🔢 Heat Wave Number (HWN)\\nNumber of heat wave events per year', fontsize=12, fontweight='bold')
    axes[0,1].set_xlabel('Year')
    axes[0,1].set_ylabel('Events')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Heat Wave Duration (HWD)
    for i, city in enumerate(cities):
        city_data = data_df[data_df['city'] == city]
        city_short = city.split(' (')[0]
        
        axes[0,2].plot(city_data['year'], city_data['HWD'], 
                      marker='^', linewidth=2, label=city_short, color=colors[i], alpha=0.7)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(city_data['year'], city_data['HWD'])
        trend_line = slope * city_data['year'] + intercept
        axes[0,2].plot(city_data['year'], trend_line, '--', color=colors[i], alpha=0.5, linewidth=1)
    
    axes[0,2].plot(country_avg.index, country_avg['HWD'], 
                  marker='v', linewidth=4, label=f'{country_name} Average', color='black', alpha=0.8)
    slope, intercept, r_value, p_value, std_err = stats.linregress(country_avg.index, country_avg['HWD'])
    trend_line = slope * country_avg.index + intercept
    axes[0,2].plot(country_avg.index, trend_line, '--', color='black', linewidth=2, alpha=0.7)
    
    axes[0,2].set_title('⏱️ Heat Wave Duration (HWD)\\nAverage duration of heat wave events', fontsize=12, fontweight='bold')
    axes[0,2].set_xlabel('Year')
    axes[0,2].set_ylabel('Days')
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Heat Wave Amplitude (HWA)
    for i, city in enumerate(cities):
        city_data = data_df[data_df['city'] == city]
        city_short = city.split(' (')[0]
        
        axes[1,0].plot(city_data['year'], city_data['HWA'], 
                      marker='d', linewidth=2, label=city_short, color=colors[i], alpha=0.7)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(city_data['year'], city_data['HWA'])
        trend_line = slope * city_data['year'] + intercept
        axes[1,0].plot(city_data['year'], trend_line, '--', color=colors[i], alpha=0.5, linewidth=1)
    
    axes[1,0].plot(country_avg.index, country_avg['HWA'], 
                  marker='p', linewidth=4, label=f'{country_name} Average', color='black', alpha=0.8)
    slope, intercept, r_value, p_value, std_err = stats.linregress(country_avg.index, country_avg['HWA'])
    trend_line = slope * country_avg.index + intercept
    axes[1,0].plot(country_avg.index, trend_line, '--', color='black', linewidth=2, alpha=0.7)
    
    axes[1,0].set_title('🔥 Heat Wave Amplitude (HWA)\\nAverage peak EHF of heat wave events', fontsize=12, fontweight='bold')
    axes[1,0].set_xlabel('Year')
    axes[1,0].set_ylabel('EHF Units')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Heat Wave Magnitude (HWM)
    for i, city in enumerate(cities):
        city_data = data_df[data_df['city'] == city]
        city_short = city.split(' (')[0]
        
        axes[1,1].plot(city_data['year'], city_data['HWM'], 
                      marker='h', linewidth=2, label=city_short, color=colors[i], alpha=0.7)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(city_data['year'], city_data['HWM'])
        trend_line = slope * city_data['year'] + intercept
        axes[1,1].plot(city_data['year'], trend_line, '--', color=colors[i], alpha=0.5, linewidth=1)
    
    axes[1,1].plot(country_avg.index, country_avg['HWM'], 
                  marker='8', linewidth=4, label=f'{country_name} Average', color='black', alpha=0.8)
    slope, intercept, r_value, p_value, std_err = stats.linregress(country_avg.index, country_avg['HWM'])
    trend_line = slope * country_avg.index + intercept
    axes[1,1].plot(country_avg.index, trend_line, '--', color='black', linewidth=2, alpha=0.7)
    
    axes[1,1].set_title('📊 Heat Wave Magnitude (HWM)\\nAverage cumulative EHF of heat wave events', fontsize=12, fontweight='bold')
    axes[1,1].set_xlabel('Year')
    axes[1,1].set_ylabel('Cumulative EHF')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Explanatory Text about EHF and All Metrics
    axes[1,2].text(0.05, 0.95, 
        """HEAT WAVE METRICS DEFINITIONS
==================================================
HWF - Heat Wave Frequency: Days per year above EHF threshold
HWN - Heat Wave Number: Number of heat wave events per year  
HWD - Heat Wave Duration: Average duration of heat wave events
HWA - Heat Wave Amplitude: Average peak EHF of heat wave events
HWM - Heat Wave Magnitude: Average cumulative EHF of heat wave events

EXCESS HEAT FACTOR (EHF) METHODOLOGY:
EHF combines acclimatization and significance indices:
• EHI_accl = T_avg(3day) - T_avg(30day)
• EHI_sig = T_avg(3day) - T95_climatology  
• EHF = max(1, EHI_accl) × EHI_sig [when EHI_sig > 0]

EHF INTERPRETATION:
• EHF 0-5: Mild heat stress
• EHF 5-15: Moderate heat stress, vulnerable groups affected
• EHF 15-30: Severe heat stress, widespread health impacts
• EHF >30: Extreme heat stress, public health emergency

CHART LEGEND:
• Solid lines: Individual city values
• Dashed lines: Linear trends
• Black line: Country average across all major cities
• Different markers distinguish the five metrics""",
        transform=axes[1,2].transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.7))
    
    axes[1,2].set_xlim(0, 1)
    axes[1,2].set_ylim(0, 1)
    axes[1,2].set_xticks([])
    axes[1,2].set_yticks([])
    axes[1,2].set_title('📚 Heat Wave Metrics Guide', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.94)
    plt.show()

def create_summary_stats(data_df):
    """Create detailed summary statistics for all five heat wave metrics"""
    
    print("\\n" + "=" * 100)
    print("📊 COMPREHENSIVE HEAT WAVE ANALYSIS SUMMARY")
    print("=" * 100)
    
    # Overall statistics
    max_hwf_row = data_df.loc[data_df['HWF'].idxmax()]
    max_hwa_row = data_df.loc[data_df['HWA'].idxmax()]
    max_hwm_row = data_df.loc[data_df['HWM'].idxmax()]
    
    print(f"🔥 EXTREME VALUES:")
    print(f"   Highest Heat Wave Frequency: {max_hwf_row['city'].split(' (')[0]} - {max_hwf_row['HWF']:.1f} days ({max_hwf_row['year']})")
    print(f"   Highest Heat Wave Amplitude:  {max_hwa_row['city'].split(' (')[0]} - {max_hwa_row['HWA']:.2f} EHF ({max_hwa_row['year']})")
    print(f"   Highest Heat Wave Magnitude:  {max_hwm_row['city'].split(' (')[0]} - {max_hwm_row['HWM']:.1f} cumulative EHF ({max_hwm_row['year']})")
    
    # City averages for all five metrics
    city_stats = data_df.groupby('city').agg({
        'HWF': 'mean',  # Heat Wave Frequency
        'HWN': 'mean',  # Heat Wave Number
        'HWD': 'mean',  # Heat Wave Duration
        'HWA': 'mean',  # Heat Wave Amplitude
        'HWM': 'mean',  # Heat Wave Magnitude
        'EHF_raw': 'mean',
        'climate': 'first'
    }).round(2)
    
    print(f"\\n📈 COMPREHENSIVE CITY AVERAGES:")
    print(f"{'City':<20} {'HWF':<6} {'HWN':<6} {'HWD':<6} {'HWA':<7} {'HWM':<7} {'Climate':<12}")
    print(f"{'':20} {'days':<6} {'evts':<6} {'days':<6} {'EHF':<7} {'cum':<7}")
    print("-" * 85)
    
    for city, stats in city_stats.iterrows():
        city_short = city.split(' (')[0]
        print(f"{city_short:<20} {stats['HWF']:<6.1f} {stats['HWN']:<6.1f} {stats['HWD']:<6.1f} {stats['HWA']:<7.2f} {stats['HWM']:<7.1f} {stats['climate']:<12}")
    
    print("\\n" + "=" * 100)

def create_country_statistics_table(country_name):
    """Create comprehensive statistics table for all cities with all five heat wave metrics"""
    
    print(f"\\n📊 Generating comprehensive statistics for all cities in {country_name}...")
    
    # Get all cities in the country
    country_cities = major_cities.filter(ee.Filter.eq('GC_CNT_GAD', country_name))
    all_cities_info = country_cities.getInfo()
    
    print(f"Processing {len(all_cities_info['features'])} cities...")
    
    # Generate analysis years (default period)
    analysis_years = list(range(2020, 2024))
    
    # Process all cities in the country
    city_summary_stats = []
    
    from scipy import stats
    
    for feature in all_cities_info['features']:
        props = feature['properties']
        city_name = props.get('GC_UCN_MAI', 'Unknown')
        population = props.get('GC_POP_TOT', 0)
        
        if city_name != 'Unknown' and population >= 100000:
            display_name = f"{city_name} ({population:,.0f})"
            
            # Extract coordinates
            geom = feature['geometry']
            if geom['type'] == 'Point':
                coords = geom['coordinates']
            elif geom['type'] == 'Polygon' and geom.get('coordinates'):
                coords_list = geom['coordinates'][0]
                if coords_list:
                    lons = [c[0] for c in coords_list]
                    lats = [c[1] for c in coords_list]
                    coords = [sum(lons)/len(lons), sum(lats)/len(lats)]
                else:
                    coords = [0, 0]
            else:
                coords = [0, 0]
            
            # Temporarily add to global city_data for heat wave calculation
            city_data[display_name] = {
                'name': city_name,
                'country': country_name,
                'population': population,
                'coords': coords,
                'feature': feature
            }
            
            # Generate heat wave data for this city
            city_df = generate_heat_wave_data(display_name, analysis_years)
            
            # Calculate summary statistics and trends for all five metrics
            hwf_values = city_df['HWF'].values
            hwn_values = city_df['HWN'].values  
            hwd_values = city_df['HWD'].values
            hwa_values = city_df['HWA'].values
            hwm_values = city_df['HWM'].values
            ehf_values = city_df['EHF_raw'].values
            
            # Calculate trends (slope per year) for all metrics
            hwf_trend, _, hwf_r2, hwf_p, _ = stats.linregress(analysis_years, hwf_values)
            hwn_trend, _, hwn_r2, hwn_p, _ = stats.linregress(analysis_years, hwn_values)
            hwd_trend, _, hwd_r2, hwd_p, _ = stats.linregress(analysis_years, hwd_values)
            hwa_trend, _, hwa_r2, hwa_p, _ = stats.linregress(analysis_years, hwa_values)
            hwm_trend, _, hwm_r2, hwm_p, _ = stats.linregress(analysis_years, hwm_values)
            ehf_trend, _, ehf_r2, ehf_p, _ = stats.linregress(analysis_years, ehf_values)
            
            city_summary_stats.append({
                # Basic city information
                'City': city_name,
                'Country': country_name,
                'Population': int(population),
                'Latitude': coords[1],
                'Longitude': coords[0],
                'Climate_Type': city_df['climate'].iloc[0],
                
                # Mean values for all five standard heat wave metrics
                'HWF_Mean': round(hwf_values.mean(), 2),  # Heat Wave Frequency
                'HWN_Mean': round(hwn_values.mean(), 2),  # Heat Wave Number
                'HWD_Mean': round(hwd_values.mean(), 2),  # Heat Wave Duration
                'HWA_Mean': round(hwa_values.mean(), 2),  # Heat Wave Amplitude
                'HWM_Mean': round(hwm_values.mean(), 2),  # Heat Wave Magnitude
                'EHF_raw_Mean': round(ehf_values.mean(), 2),
                
                # Standard deviations
                'HWF_Std': round(hwf_values.std(), 2),
                'HWN_Std': round(hwn_values.std(), 2),
                'HWD_Std': round(hwd_values.std(), 2),
                'HWA_Std': round(hwa_values.std(), 2),
                'HWM_Std': round(hwm_values.std(), 2),
                'EHF_raw_Std': round(ehf_values.std(), 2),
                
                # Min/Max values
                'HWF_Min': round(hwf_values.min(), 2),
                'HWF_Max': round(hwf_values.max(), 2),
                'HWN_Min': round(hwn_values.min(), 2),
                'HWN_Max': round(hwn_values.max(), 2),
                'HWD_Min': round(hwd_values.min(), 2),
                'HWD_Max': round(hwd_values.max(), 2),
                'HWA_Min': round(hwa_values.min(), 2),
                'HWA_Max': round(hwa_values.max(), 2),
                'HWM_Min': round(hwm_values.min(), 2),
                'HWM_Max': round(hwm_values.max(), 2),
                
                # Trends (per decade) for all five metrics
                'HWF_Trend_per_decade': round(hwf_trend * 10, 2),
                'HWN_Trend_per_decade': round(hwn_trend * 10, 2),
                'HWD_Trend_per_decade': round(hwd_trend * 10, 2),
                'HWA_Trend_per_decade': round(hwa_trend * 10, 2),
                'HWM_Trend_per_decade': round(hwm_trend * 10, 2),
                'EHF_Trend_per_decade': round(ehf_trend * 10, 2),
                
                # Trend significance (R² and p-values)
                'HWF_Trend_R2': round(hwf_r2, 3),
                'HWN_Trend_R2': round(hwn_r2, 3),
                'HWD_Trend_R2': round(hwd_r2, 3),
                'HWA_Trend_R2': round(hwa_r2, 3),
                'HWM_Trend_R2': round(hwm_r2, 3),
                'HWF_Trend_pvalue': round(hwf_p, 3),
                'HWN_Trend_pvalue': round(hwn_p, 3),
                'HWD_Trend_pvalue': round(hwd_p, 3),
                'HWA_Trend_pvalue': round(hwa_p, 3),
                'HWM_Trend_pvalue': round(hwm_p, 3)
            })
    
    # Create DataFrame
    summary_df = pd.DataFrame(city_summary_stats)
    
    # Display summary
    print(f"\\n✅ Generated statistics for {len(summary_df)} cities in {country_name}")
    print(f"\\n📋 Preview of Comprehensive Heat Wave Statistics:")
    print("=" * 140)
    
    # Show preview table with all five metrics
    preview_cols = ['City', 'Population', 'Climate_Type', 'HWF_Mean', 'HWN_Mean', 'HWD_Mean', 'HWA_Mean', 'HWM_Mean', 'HWF_Trend_per_decade', 'HWA_Trend_per_decade']
    preview_df = summary_df[preview_cols].head(10)
    print(preview_df.to_string(index=False))
    
    if len(summary_df) > 10:
        print(f"\\n... and {len(summary_df) - 10} more cities (see full CSV download)")
    
    # Create download link
    import base64
    csv_content = summary_df.to_csv(index=False)
    b64_csv = base64.b64encode(csv_content.encode()).decode()
    
    filename = f"{country_name.replace(' ', '_')}_comprehensive_heat_wave_statistics_{analysis_years[0]}_{analysis_years[-1]}.csv"
    
    download_link = f'''
    <div style="margin: 20px 0; padding: 15px; background-color: #f0f8ff; border-radius: 5px; border: 2px solid #4CAF50;">
        <h3 style="color: #2E7D32; margin-top: 0;">📁 Download Complete Heat Wave Dataset</h3>
        <p><strong>File:</strong> {filename}</p>
        <p><strong>Contains:</strong> {len(summary_df)} cities with {len(summary_df.columns)} variables</p>
        <p><strong>All Five Standard Heat Wave Metrics:</strong></p>
        <ul>
            <li><strong>HWF</strong> - Heat Wave Frequency: Days per year above EHF threshold</li>
            <li><strong>HWN</strong> - Heat Wave Number: Number of heat wave events per year</li>
            <li><strong>HWD</strong> - Heat Wave Duration: Average duration of heat wave events</li>
            <li><strong>HWA</strong> - Heat Wave Amplitude: Average peak EHF of heat wave events</li>
            <li><strong>HWM</strong> - Heat Wave Magnitude: Average cumulative EHF of heat wave events</li>
        </ul>
        <p><strong>Statistics:</strong> Mean, standard deviation, min/max values</p>
        <p><strong>Trends:</strong> Per-decade trends with R² and p-values for all metrics</p>
        <a href="data:text/csv;base64,{b64_csv}" download="{filename}" 
           style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 4px; font-weight: bold;">
           📥 Download Comprehensive CSV File
        </a>
    </div>
    '''
    
    display(widgets.HTML(download_link))
    
    print(f"\\n📊 COMPREHENSIVE COUNTRY SUMMARY - {country_name}")
    print("=" * 80)
    print(f"Total cities analyzed: {len(summary_df)}")
    print(f"\\nAVERAGE VALUES ACROSS ALL CITIES:")
    print(f"• HWF (Frequency): {summary_df['HWF_Mean'].mean():.1f} ± {summary_df['HWF_Mean'].std():.1f} days/year")
    print(f"• HWN (Number): {summary_df['HWN_Mean'].mean():.1f} ± {summary_df['HWN_Mean'].std():.1f} events/year")
    print(f"• HWD (Duration): {summary_df['HWD_Mean'].mean():.1f} ± {summary_df['HWD_Mean'].std():.1f} days")
    print(f"• HWA (Amplitude): {summary_df['HWA_Mean'].mean():.2f} ± {summary_df['HWA_Mean'].std():.2f} EHF units")
    print(f"• HWM (Magnitude): {summary_df['HWM_Mean'].mean():.1f} ± {summary_df['HWM_Mean'].std():.1f} cumulative EHF")
    
    print(f"\\nCOUNTRY-WIDE TRENDS (per decade):")
    print(f"• HWF Trend: {summary_df['HWF_Trend_per_decade'].mean():.1f} days/decade")
    print(f"• HWN Trend: {summary_df['HWN_Trend_per_decade'].mean():.1f} events/decade")
    print(f"• HWD Trend: {summary_df['HWD_Trend_per_decade'].mean():.1f} days/decade")
    print(f"• HWA Trend: {summary_df['HWA_Trend_per_decade'].mean():.2f} EHF units/decade")
    print(f"• HWM Trend: {summary_df['HWM_Trend_per_decade'].mean():.1f} cumulative EHF/decade")
    
    # Risk assessment based on multiple metrics
    high_risk_hwf = len(summary_df[summary_df['HWF_Mean'] > 40])  # High frequency
    high_risk_hwa = len(summary_df[summary_df['HWA_Mean'] > 10])  # High amplitude
    high_risk_hwm = len(summary_df[summary_df['HWM_Mean'] > 60])  # High magnitude
    
    print(f"\\nRISK ASSESSMENT:")
    print(f"🔥 High frequency cities (HWF > 40 days): {high_risk_hwf} ({high_risk_hwf/len(summary_df)*100:.1f}%)")
    print(f"🔥 High amplitude cities (HWA > 10 EHF): {high_risk_hwa} ({high_risk_hwa/len(summary_df)*100:.1f}%)")  
    print(f"🔥 High magnitude cities (HWM > 60): {high_risk_hwm} ({high_risk_hwm/len(summary_df)*100:.1f}%)")
    
    return summary_df

def load_cities_for_country(country_name):
    """Load cities for the selected country"""
    global city_data, current_cities
    
    print(f"Loading cities for {country_name}...")
    
    # Filter cities by country
    country_cities = major_cities.filter(ee.Filter.eq('GC_CNT_GAD', country_name))
    cities_info = country_cities.getInfo()
    
    # Process city data
    city_options = []
    city_data = {}
    
    for feature in cities_info['features']:
        props = feature['properties']
        city_name = props.get('GC_UCN_MAI', 'Unknown')
        population = props.get('GC_POP_TOT', 0)
        
        if city_name != 'Unknown':
            display_name = f"{city_name} ({population:,.0f})"
            city_options.append(display_name)
            
            # Extract coordinates efficiently
            geom = feature['geometry']
            if geom['type'] == 'Point':
                coords = geom['coordinates']
            elif geom['type'] == 'Polygon' and geom.get('coordinates'):
                coords_list = geom['coordinates'][0]
                if coords_list:
                    lons = [c[0] for c in coords_list]
                    lats = [c[1] for c in coords_list]
                    coords = [sum(lons)/len(lons), sum(lats)/len(lats)]
                else:
                    coords = [0, 0]
            else:
                coords = [0, 0]
            
            city_data[display_name] = {
                'name': city_name,
                'country': country_name,
                'population': population,
                'coords': coords,
                'feature': feature
            }
    
    current_cities = sorted(city_options)
    print(f"✅ Loaded {len(current_cities)} cities for {country_name}")
    return current_cities

print("✅ All analysis functions defined with comprehensive 5-metric heat wave analysis")

✅ All analysis functions defined with comprehensive 5-metric heat wave analysis


## Interactive Interface and Analysis

In [4]:
# Create selection widgets
country_selector = widgets.Dropdown(
    options=['Select Country...'] + countries,
    value='Select Country...',
    description='Country:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

city_selector = widgets.SelectMultiple(
    options=[],
    description='Select Cities:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(height='200px', width='600px'),
    disabled=True
)

analysis_years = widgets.IntRangeSlider(
    value=[2020, 2023],
    min=2000,
    max=2024,
    step=1,
    description='Analysis Years:',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='🔥 Run Heat Wave Analysis',
    button_style='success',
    layout=widgets.Layout(width='300px', height='40px'),
    disabled=True
)

country_stats_button = widgets.Button(
    description='📊 Generate Country Statistics',
    button_style='info',
    layout=widgets.Layout(width='300px', height='40px'),
    disabled=True
)

output = widgets.Output()

# Country selection handler
def on_country_change(change):
    if change['new'] != 'Select Country...':
        with output:
            clear_output(wait=True)
            cities_for_country = load_cities_for_country(change['new'])
            city_selector.options = cities_for_country
            city_selector.disabled = False
            run_button.disabled = False
            country_stats_button.disabled = False

country_selector.observe(on_country_change, names='value')

# Analysis function
def run_analysis(button):
    """Main analysis function with enhanced visualizations"""
    with output:
        clear_output(wait=True)
        
        selected_cities = list(city_selector.value)
        years = list(range(analysis_years.value[0], analysis_years.value[1] + 1))
        country = country_selector.value
        
        if country == 'Select Country...':
            print("⚠️ Please select a country first")
            return
            
        if not selected_cities:
            print("⚠️ Please select at least one city")
            return
        
        if len(selected_cities) > 10:
            print("⚠️ Please select 10 or fewer cities for optimal visualization")
            return
        
        print(f"🔄 Analyzing {len(selected_cities)} cities in {country}")
        print(f"   Years: {years[0]}-{years[-1]} ({len(years)} years)")
        print(f"   Cities: {', '.join([c.split(' (')[0] for c in selected_cities])}")
        print("\\n" + "-" * 50)
        
        # Generate data for all cities
        all_data = []
        for city_display_name in selected_cities:
            city_df = generate_heat_wave_data(city_display_name, years)
            all_data.append(city_df)
        
        combined_data = pd.concat(all_data, ignore_index=True)
        
        # Create enhanced visualizations
        create_charts(combined_data, country)
        create_summary_stats(combined_data)
        
        print("\\n✅ Selected cities analysis complete!")
        print("\\nℹ️  Use 'Generate Country Statistics' button for comprehensive country-wide data with CSV download.")

# Country statistics function
def generate_country_stats(button):
    """Generate comprehensive country statistics with download"""
    with output:
        clear_output(wait=True)
        
        country = country_selector.value
        
        if country == 'Select Country...':
            print("⚠️ Please select a country first")
            return
        
        print(f"🌍 Generating comprehensive heat wave statistics for {country}...")
        print("This will analyze ALL major cities (100k+ population) in the country.")
        print("Please wait, this may take a moment...")
        
        # Generate comprehensive country statistics
        country_df = create_country_statistics_table(country)
        
        print("\\n✅ Country statistics generation complete!")

# Connect the buttons
run_button.on_click(run_analysis)
country_stats_button.on_click(generate_country_stats)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h3>🏙️ Country-City Heat Wave Analysis</h3>"),
    widgets.HTML("<p><b>Step 1:</b> Select a country from the dropdown</p>"),
    country_selector,
    widgets.HTML("<p><b>Step 2:</b> Choose analysis type:</p>"),
    widgets.HTML("<p><b>Option A:</b> Select specific cities for detailed comparison</p>"),
    city_selector,
    analysis_years,
    run_button,
    widgets.HTML("<p><b>Option B:</b> Generate complete country statistics with CSV download</p>"),
    country_stats_button,
    output
]))

print("🎯 Select a country to begin heat wave analysis!")

🎯 Select a country to begin heat wave analysis!


## Map and Analysis Complete

The notebook is ready for heat wave analysis. All functions are properly defined and the interface is active.

In [48]:
print("✅ All functions loaded and interface is ready!")

✅ All functions loaded and interface is ready!
